# MNIST Handwritten Digit Classifier

This notebook demonstrates how to build a neural network to classify handwritten digits (0-9) from the MNIST dataset using TensorFlow and Keras.

## Requirements
1. Load MNIST dataset
2. Normalize data
3. Build Neural Network (Flatten, Dense, ReLU, Softmax)
4. Compile (Adam, Categorical Crossentropy, Accuracy)
5. Train for 5 epochs
6. Evaluate and Visualize Results

In [ ]:
# Install necessary libraries if not already installed
!pip install tensorflow numpy matplotlib seaborn scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")

## 1. Load and Preprocess Data

- **Load**: Get the MNIST dataset (60k train, 10k test).
- **Normalize**: Scale pixel values from [0, 255] to [0, 1].
- **Reshape/Format**: Ensure data shape is correct and labels are categorical.

In [ ]:
# Load the data
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize pixel values to be between 0 and 1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# One-hot encode the labels
num_classes = 10
y_train_cat = keras.utils.to_categorical(y_train, num_classes)
y_test_cat = keras.utils.to_categorical(y_test, num_classes)

## 2. Build the Model

We will build a simple Feed-Forward Neural Network (MLP).
- **Flatten Layer**: Converts 28x28 images to a 1D vector.
- **Dense Hidden Layer**: 128 neurons with ReLU activation.
- **Output Layer**: 10 neurons (one for each digit) with Softmax activation.

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

## 3. Compile the Model

- **Optimizer**: Adam (Standard choice for most problems)
- **Loss Function**: Categorical Crossentropy (since we used one-hot encoding)
- **Metric**: Accuracy

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

## 4. Train the Model

Training for 5 epochs with a batch size of 128. We also use 10% of the training data for validation.

In [ ]:
batch_size = 128
epochs = 5

history = model.fit(x_train, y_train_cat, batch_size=batch_size, epochs=epochs, validation_split=0.1)

## 5. Evaluate Model Performance

We will evaluate the model on the unseen test dataset.

In [ ]:
score = model.evaluate(x_test, y_test_cat, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

### Plot Training History
Visualizing the accuracy and loss over epochs.

In [ ]:
plt.figure(figsize=(12, 4))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='lower right')

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.show()

### Detailed Metrics & Confusion Matrix
Calculating Precision, Recall, and F1 Score.

In [ ]:
# Predict the classes
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='macro')
recall = recall_score(y_test, y_pred_classes, average='macro')
f1 = f1_score(y_test, y_pred_classes, average='macro')

print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred_classes))

In [ ]:
# Generate Confusion Matrix
cm = confusion_matrix(y_test, y_pred_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()